In [1]:
# Load Image

# Editor appearance set up & Load plot & Calculate DGCI

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import packages needed
import gc
import pickle
import platform
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.widgets import PolygonSelector
import numpy as np
import pandas as pd
from skimage import io, draw
from ipywidgets import widgets
from osgeo import gdal

import general_funcs


# OS related settings
if platform.system() == 'Windows':
    print('Windows')
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Load image and print size & pre-process
# Use skimage to load multi-layer tiff file

image_file = askopenfilename(title='Load image file', initialdir='./data/field_image')
plot_loc_file = askopenfilename(title='Load plot location file', initialdir='./data/plot_location')

img = io.imread(image_file)
print("Original Image Shape: ", img.shape)

# Load GPS coordinate from file & Calculate pixel location
try:
    with open(plot_loc_file, 'rb') as f:
        interested_area = pickle.load(f)
        plot_vertices_gps = pickle.load(f)
        plot_notes = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

# Trim to area of interest
ul = np.min(interested_area, 1)
br = np.max(interested_area, 1)
img = img[ul[1]:br[1], ul[0]:br[0], :]
print("Trimmed Image Shape: ", img.shape)

# Calculating pixel location from GPS coordinate
ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
plot_vertices = general_funcs.plotVGPS2plotV(plot_vertices_gps, gt)
all_vertices = np.concatenate(list(plot_vertices.values()), axis=0)

# Extract layers from the multilayer tiff file and do some adjustments
scale_factor = 3
h, w, d = img.shape
layer_RGB, layer_IR, layer_mask = general_funcs.extract_layers(img)
if d != 2:
    layer_RGB_low_res = general_funcs.low_res(layer_RGB, scale_factor)

layer_temp = np.zeros(layer_IR.shape)
if layer_IR.max() > int('0b' + '1100000000000000', 2):
    # 16 bit to 14 bit + nonlinear temp calculation
    layer_temp[np.where(layer_mask!=0)] = general_funcs.flir_non_linear_thermal_to_temp(layer_IR[np.where(layer_mask!=0)] - int('0b' + '1100000000000000', 2))
else:
    layer_temp[np.where(layer_mask!=0)] = general_funcs.flir_linear_high_res_thermal_to_temp(layer_IR[np.where(layer_mask!=0)])




# RGB to HSV (H 0-360, S 0-1, V 0-255)
if d == 5 or d == 4:
    layer_HSV = general_funcs.RGB2HSV(layer_RGB)
    # Calculate Vegetation Index
    layer_DGCI = general_funcs.DGCI(layer_HSV)
    print('DGCI calculated')

# Remove original img file to save space in memory
del(img)

macOS
Interactive plot activated
Original Image Shape:  (8315, 3668, 2)
Trimmed Image Shape:  (7513, 3256, 2)


In [2]:
# Show Image

def update_transparency(slider_value):
    global transparency
    transparency = slider_value
    
def update_hue_mask(button):
    button.description = 'Update Hue Mask (Updating)'
    global hue_range_updated
    if hue_range_updated:
        hue_mask[np.where((layer_hue>=hue_range[0]) * (layer_hue<=hue_range[1]))] = 1
        hue_mask[np.where(np.invert((layer_hue>=hue_range[0]) * (layer_hue<=hue_range[1])))] = 0
        hue_range_updated = False
    red_masked_RGB = red_mask * hue_mask * transparency + layer_RGB * (1 - hue_mask * transparency)
    red_masked_RGB = red_masked_RGB.astype(np.uint8)
    ax.imshow(red_masked_RGB)
    
    a, b, c = hue_mask.shape
    canopy = hue_mask[:, :, 0].sum()
    canopy_closure = canopy/(a*b)
    canopy_closure = "{:.2%}".format(canopy_closure)
    canopy_closure_text.value = canopy_closure
    
    button.description = 'Update Hue Mask (Done)'
    
def update_hue_range(slider_value):
    global hue_range, hue_range_updated
    hue_range = np.array(slider_value)
    hue_range_updated = True
    
def save_hue_range(button):
    lon_meter_per_pix, lat_meter_per_pix = general_funcs.meter_per_pix(gt)
    size_per_pix = lon_meter_per_pix * lat_meter_per_pix
    
    for_flag = 0
    for plot_name in plot_vertices.keys():
        for_flag += 1
        if for_flag % 10 == 1:
            print('Calculating plot No.', for_flag,  plot_name)
        one_plot_vertices = plot_vertices[plot_name]
        one_plot_vertices_transformed = one_plot_vertices - ul
        rr, cc = draw.polygon(one_plot_vertices_transformed[:, 1], one_plot_vertices_transformed[:, 0], layer_mask.shape)
        rr, cc = rr.astype(np.uint16), cc.astype(np.uint16)
        plot_mask = np.zeros(layer_mask.shape).astype(np.int8)
        plot_mask[rr, cc] = 1
        inds = np.where(plot_mask != 0)
        inds = (inds[0].astype(np.uint16), inds[1].astype(np.uint16))
        
        plot_areas.append(size_per_pix * rr.size)
        hue_ranges.append(hue_range)
        
#         Calculate indices
        avg_temp = np.round(np.average(layer_temp[inds]), 2)
        avg_temps.append(avg_temp)

        max_temp = np.round(np.max(layer_temp[inds]), 2)
        max_temps.append(max_temp)
        
        min_temp = np.round(np.min(layer_temp[inds]), 2)
        min_temps.append(min_temp)

        if d == 5:
            avg_DGCI = np.average(layer_DGCI[inds])
            avg_DGCI = np.round(avg_DGCI, 2)
            avg_DGCIs.append(avg_DGCI)

            max_DGCI = np.max(layer_DGCI[inds])
            max_DGCIs.append(max_DGCI)

            min_DGCI = np.min(layer_DGCI[inds])
            min_DGCIs.append(min_DGCI)

            avg_RGB = np.mean(layer_RGB[inds], axis=0)
            avg_RGB = np.round(avg_RGB, 2)
            avg_RGBs.append(avg_RGB)

            avg_HSV = np.mean(layer_HSV[inds], axis=0)
            avg_HSV = np.round(avg_HSV, 2)
            avg_HSVs.append(avg_HSV)

            # Apply hue mask

            mask_hue_restrict = plot_mask * hue_mask[:, :, 0]
            inds = np.where(mask_hue_restrict == 1)

            cnp_cls = mask_hue_restrict.sum()/plot_mask.sum()
            cnp_cls = "{:.2%}".format(cnp_cls)
            canopy_closures.append(cnp_cls)
        
            if inds[0].size != 0:
                avg_DGCI = np.average(layer_DGCI[inds])
                avg_DGCI = np.round(avg_DGCI, 2)
                avg_DGCIs_hue_restrict.append(avg_DGCI)

                max_DGCI = np.max(layer_DGCI[inds])
                max_DGCIs_hue_restrict.append(max_DGCI)

                min_DGCI = np.min(layer_DGCI[inds])
                min_DGCIs_hue_restrict.append(min_DGCI)

                avg_RGB = np.mean(layer_RGB[inds], axis=0)
                avg_RGB = np.round(avg_RGB, 2)
                avg_RGBs_hue_restrict.append(avg_RGB)

                avg_HSV = np.mean(layer_HSV[inds], axis=0)
                avg_HSV = np.round(avg_HSV, 2)
                avg_HSVs_hue_restrict.append(avg_HSV)

#                 avg_IR = np.average(layer_IR[inds])
                avg_temp = np.round(np.average(layer_temp[inds]), 2)
                avg_temps_hue_restrict.append(avg_temp)

#                 max_IR = np.max(layer_IR[inds])
                max_temp = np.round(np.max(layer_temp[inds]), 2)
                max_temps_hue_restrict.append(max_temp)

#                 min_IR = np.min(layer_IR[inds])
                min_temp = np.round(np.min(layer_temp[inds]), 2)
                min_temps_hue_restrict.append(min_temp)

            else:
                avg_DGCIs_hue_restrict.append(0)
                max_DGCIs_hue_restrict.append(0)
                min_DGCIs_hue_restrict.append(0)
                avg_RGBs_hue_restrict.append(np.asarray([0, 0, 0]))
                avg_HSVs_hue_restrict.append(np.asarray([0, 0, 0]))
                avg_temps_hue_restrict.append(0)
                max_temps_hue_restrict.append(0)
                min_temps_hue_restrict.append(0)
        
        
    if d != 4:
        vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)
    if d == 5 or d == 4:
#         ax.imshow(layer_RGB)
        ;
    elif d == 2:
#         myax = ax.imshow(layer_IR, cmap='gist_gray', vmin=vmin, vmax=vmax)
        ;
    
    all_widgets.children = [show_RGB_button, show_IR_button, show_DGCIImage_button, show_temp_button, show_DGCI_button, show_canopy_closure_button, save_button]
    

    
    
# Show indices of plots

def show_RGB(button):
    ax.imshow(layer_RGB)
    
def show_IR(button):
    axim = ax.imshow(layer_IR, cmap=plt.get_cmap(color_map))
    
def show_DGCI_image(button):
    axim = ax.imshow(layer_DGCI, cmap=plt.get_cmap(color_map))
    
def show_temp(button):
    if button.description == 'Show Average Temperature':
        for plot_name in plot_vertices.keys():
            one_plot_vertices = plot_vertices[plot_name]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_name + '\n' + str(avg_temps[int(list(plot_vertices.keys()).index(plot_name))]) + '℃', ha='center', va='center')
            
        button.description = 'Hide Average Temperature'
    
    elif button.description == 'Hide Average Temperature':
        ax.patches.clear()
        ax.texts.clear()
        button.description = 'Show Average Temperature'
        plt.show()
    
def show_DGCI(button):
    if button.description == 'Show Average DGCI': 
        for plot_name in plot_vertices.keys():
            one_plot_vertices = plot_vertices[plot_name]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_name + '\n' + str(avg_DGCIs[int(list(plot_vertices.keys()).index(plot_name))]), ha='center', va='center')
            
        button.description = 'Hide Average DGCI'
    
    elif button.description == 'Hide Average DGCI':
        ax.patches.clear()
        ax.texts.clear()
        button.description = 'Show Average DGCI'
#         plt.show()
        
def show_canopy_closure(button):
    if button.description == 'Show Canopy Closure':
        for plot_name in plot_vertices.keys():
            one_plot_vertices = plot_vertices[plot_name]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_name + '\n' + str(canopy_closures[int(list(plot_vertices.keys()).index(plot_name))]), ha='center', va='center')
            
        button.description = 'Hide Canopy Closure'
    
    elif button.description == 'Hide Canopy Closure':
        ax.patches.clear()
        ax.texts.clear()
        button.description = 'Show Canopy Closure'
        
def save_result(button):
    global avg_temps, max_temps, min_temps, avg_DGCIs, max_DGCIs, min_DGCIs, avg_RGBs, avg_HSVs, canopy_closures, plot_areas, hue_ranges
    global avg_DGCIs_hue_restrict, max_DGCIs_hue_restrict, min_DGCIs_hue_restrict, avg_RGBs_hue_restrict, avg_HSVs_hue_restrict, avg_temps_hue_restrict, max_temps_hue_restrict, min_temps_hue_restrict
    
    plot_areas = np.array(plot_areas)
    hue_ranges = np.array(hue_ranges)
    if d != 4:
        avg_temps = np.array(avg_temps)
        max_temps = np.array(max_temps)
        min_temps = np.array(min_temps)

    if d != 2:
        avg_DGCIs = np.array(avg_DGCIs)
        max_DGCIs = np.array(max_DGCIs)
        min_DGCIs = np.array(min_DGCIs)
        avg_RGBs = np.array(avg_RGBs)
        avg_HSVs = np.array(avg_HSVs)

        avg_DGCIs_hue_restrict = np.array(avg_DGCIs_hue_restrict)
        max_DGCIs_hue_restrict = np.array(max_DGCIs_hue_restrict)
        min_DGCIs_hue_restrict = np.array(min_DGCIs_hue_restrict)
        avg_RGBs_hue_restrict = np.array(avg_RGBs_hue_restrict)
        avg_HSVs_hue_restrict = np.array(avg_HSVs_hue_restrict)

        canopy_closures = np.array(canopy_closures)

    if d == 5:
        avg_temps_hue_restrict = np.array(avg_temps_hue_restrict)
        max_temps_hue_restrict = np.array(max_temps_hue_restrict)
        min_temps_hue_restrict = np.array(min_temps_hue_restrict)


    if d == 2:
        df = pd.DataFrame(data=np.column_stack((list(plot_vertices.keys()), avg_temps, max_temps, min_temps, plot_areas, list(plot_notes.values()))),
                          columns=['Plot Name', 'Avg Temp', 'Max Temp', 'Min Temp', 'Plot Area', 'Plot Notes'])
#     elif d == 4:
#         df = pd.DataFrame(data=np.column_stack((list(plot_vertices.keys()), avg_DGCIs, avg_DGCIs_hue_restrict, max_DGCIs, max_DGCIs_hue_restrict, min_DGCIs, min_DGCIs_hue_restrict, canopy_closures, avg_RGBs, avg_RGBs_hue_restrict, avg_HSVs, avg_HSVs_hue_restrict, plot_areas, hue_ranges)),
#                           columns=['Plot Name', 'Avg DGCI', 'Avg DGCI (hue restrict)', 'Max DGCI', 'Max DGCI (hue restrict)', 'Min DGCI', 'Min DGCI (hue restrict)', 'Canopy Closure', 'Avg R', 'Avg G', 'Avg B', 'Avg R (hue restrict)', 'Avg G (hue restrict)', 'Avg B (hue restrict)', 'Avg H', 'Avg S', 'Avg V', 'Avg H (hue restrict)', 'Avg S (hue restrict)', 'Avg V (hue restrict)', 'Plot Area', 'Hue Range Start', 'Hue Range End'])
    elif d == 5:
        df = pd.DataFrame(data=np.column_stack((list(plot_vertices.keys()), avg_temps, avg_temps_hue_restrict, max_temps, max_temps_hue_restrict, min_temps, min_temps_hue_restrict, avg_DGCIs, avg_DGCIs_hue_restrict, max_DGCIs, max_DGCIs_hue_restrict, min_DGCIs, min_DGCIs_hue_restrict, canopy_closures, avg_RGBs, avg_RGBs_hue_restrict, avg_HSVs, avg_HSVs_hue_restrict, plot_areas, hue_ranges, list(plot_notes.values()))),
                          columns=['Plot Name', 'Avg Temp', 'Avg Temp (hue restrict)', 'Max Temp', 'Max Temp (hue restrict)', 'Min Temp', 'Min Temp (hue restrict)', 'Avg DGCI', 'Avg DGCI (hue restrict)', 'Max DGCI', 'Max DGCI (hue restrict)', 'Min DGCI', 'Min DGCI (hue restrict)', 'Canopy Closure', 'Avg R', 'Avg G', 'Avg B', 'Avg R (hue restrict)', 'Avg G (hue restrict)', 'Avg B (hue restrict)', 'Avg H', 'Avg S', 'Avg V', 'Avg H (hue restrict)', 'Avg S (hue restrict)', 'Avg V (hue restrict)', 'Plot Area', 'Hue Range Start', 'Hue Range End', 'Plot Notes'])

    
    fn = image_file.split('/')[-2] + '_' + image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('csv', '*.csv')], title='Save Indices', initialfile=fn+'_indices', initialdir='./data/final_result/')
    if not file_name:
        return
    if not file_name.endswith('.csv'):
        file_name += '.csv'

    try:
        df.to_csv(file_name)
        print('Indices saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))


        
        
        
        
        
hue_range = [60, 180]
hue_range_updated = True
transparency = 0.5

if d != 2:
    layer_hue = layer_HSV[:, :, 0]
    hue_mask = np.zeros(layer_RGB.shape).astype(np.uint8)
    red_mask = (np.ones(hue_mask.shape) * (255, 0, 0)).astype(np.uint8)
    hue_mask_low_res = np.zeros(layer_RGB_low_res.shape).astype(np.uint8)
    red_mask_low_res = (np.ones(hue_mask_low_res.shape) * (255, 0, 0)).astype(np.uint8)


avg_temps = []
max_temps = []
min_temps = []
avg_DGCIs = []
max_DGCIs = []
min_DGCIs = []
avg_RGBs = []
avg_HSVs = []
pixel_num = []

avg_temps_hue_restrict = []
max_temps_hue_restrict = []
min_temps_hue_restrict = []
avg_DGCIs_hue_restrict = []
max_DGCIs_hue_restrict = []
min_DGCIs_hue_restrict = []
avg_RGBs_hue_restrict = []
avg_HSVs_hue_restrict = []
pixel_num_hue_restrict = []

canopy_closures = []
plot_areas = []
hue_ranges = []


# Widgets
style = {'description_width': 'initial'}
slider_layout = widgets.Layout(width='99%')
hue_slider = widgets.FloatRangeSlider(value=hue_range, min=0, max=360, step=0.01, continuous_update=False, description='Hue Range', layout=slider_layout, style=style)
hue_interactive = widgets.interactive(update_hue_range, slider_value=hue_slider)
transparency_slider = widgets.FloatSlider(value=transparency, min=0, max=1, continuous_update=False, description='Mask Transparency', readout_format='.1f', layout=slider_layout, style=style)
transparency_interactive = widgets.interactive(update_transparency, slider_value=transparency_slider)

# Button widgets
button_layout = widgets.Layout(align='center', width='100%')
canopy_closure_text = widgets.Text(value='0', description='Canopy Closure', layout=button_layout, disabled=True, style=style)
update_hue_mask_button = widgets.Button(description='Apply Hue Mask', layout=button_layout)
save_hue_range_button = widgets.Button(description='Next', layout=button_layout, button_style='success')

update_hue_mask_button.on_click(update_hue_mask)
save_hue_range_button.on_click(save_hue_range)

show_RGB_button = widgets.Button(description='Show RGB Image', layout=button_layout)
show_IR_button = widgets.Button(description='Show IR Image', layout=button_layout)
show_DGCIImage_button = widgets.Button(description='Show DGCI Image', layout=button_layout)
show_temp_button = widgets.Button(description='Show Average Temperature', layout=button_layout)
show_DGCI_button = widgets.Button(description='Show Average DGCI', layout=button_layout)
show_canopy_closure_button = widgets.Button(description='Show Canopy Closure', layout=button_layout)
save_button = widgets.Button(description='Save Result', layout=button_layout, button_style='success')

show_RGB_button.on_click(show_RGB)
show_IR_button.on_click(show_IR)
show_DGCIImage_button.on_click(show_DGCI_image)
show_temp_button.on_click(show_temp)
show_DGCI_button.on_click(show_DGCI)
show_canopy_closure_button.on_click(show_canopy_closure)
save_button.on_click(save_result)

# Box widgets
box_layout = widgets.Layout(width='100%')
button_set1 = widgets.HBox(children=[update_hue_mask_button, save_hue_range_button], layout=box_layout)
all_widgets = widgets.VBox(children=[transparency_slider, hue_slider, button_set1, canopy_closure_text], layout=box_layout)
display(all_widgets)


out = widgets.Output()
display(out)
with out:
    plt.close('all')
    # Histogram of hue value
    
    fig, ax = plt.subplots(figsize=(7, 7))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    
    if d != 2:
        fig_hist, ax_hist = plt.subplots(figsize=(5, 5))
        flattened_layer_hue = layer_hue.flatten()
        flattened_layer_mask = layer_mask.flatten()
        flattened_layer_hue = flattened_layer_hue[flattened_layer_mask != 0]
        ax_hist.hist(flattened_layer_hue, bins=100)
        mean = np.mean(flattened_layer_hue)
        var = np.var(flattened_layer_hue)
        std = np.std(flattened_layer_hue)
        print('Mean:', mean)
        print('Variance:', var)
        print('Standard Deviation', std)

        # Plot image with hue mask
        
        ax.imshow(layer_RGB_low_res)
        plt.show()
        
    else:
#         mask_not_0_inds = np.where(layer_mask > 0)
        vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)
        myax = ax.imshow(layer_IR, cmap='gist_gray', vmin=vmin, vmax=vmax)


Output()

Calculating plot No. 1 1001
Calculating plot No. 11 1194
Calculating plot No. 21 2175
Calculating plot No. 31 3156
Calculating plot No. 41 1138
Calculating plot No. 51 2119
Calculating plot No. 61 3100
Calculating plot No. 71 1082
Calculating plot No. 81 2063
Calculating plot No. 91 3043
Calculating plot No. 101 1024
Calculating plot No. 111 2004
Calculating plot No. 121 2197
Calculating plot No. 131 3178
Calculating plot No. 141 1160
Calculating plot No. 151 2141
Calculating plot No. 161 3122
Calculating plot No. 171 1104
Calculating plot No. 181 2085
Calculating plot No. 191 3066
Calculating plot No. 201 1047
Calculating plot No. 211 2027
Calculating plot No. 221 3007
Calculating plot No. 231 3200
Calculating plot No. 241 1182
Calculating plot No. 251 2163
Calculating plot No. 261 3144
Calculating plot No. 271 1126
Calculating plot No. 281 2107
Calculating plot No. 291 3088
Calculating plot No. 301 1070
Calculating plot No. 311 2050
Calculating plot No. 321 3030
Calculating plot No. 